In [24]:
import pandas as pd
import numpy as np


In [25]:
df= pd.read_csv('../Cleaned_Data/cleaned_data.csv')

In [26]:
df

,title,content,type
0,5,"Sách rất hay. Dựa trên nghiên cứu, và có hướng...",0
1,5,Nội dung sách quá hay Rất đơn giản và dễ áp dụ...,0
2,5,Quyển này hay mà tác giả viết rất dễ đọc nha n...,0
3,5,Dạo này thói quen xấu lại bủa vây nên tôi quyế...,0
4,5,"Tiki giao thần tốc, mình đặt 1h sáng giao chiề...",0
...,...,...,...
7968,4,Ship về bị chảy ra ngoài,25
7969,4,chưa dùng sp nên chưa đánh giá dc,25
7970,4,bọt ít k sạch dầu mỡ,25
7971,4,Giá hơi cao,25


# Hệ thống Dựa trên Luật (Rule-Based Systems) cho việc phân loại khía cạnh

Trong phần này, chúng ta sẽ triển khai một hệ thống dựa trên luật đơn giản để phân loại các đánh giá theo các khía cạnh khác nhau:
- Vận chuyển
- Chất lượng
- Giá cả
- Chăm sóc khách hàng (CSKH)

Mỗi luật sẽ kiểm tra sự xuất hiện của các từ khóa cụ thể trong nội dung đánh giá.

In [27]:
van_chuyen_keywords = [
    'giao hàng', 'ship', 'shipper', 'vận chuyển', 'giao', 'nhận hàng', 'đóng gói',
    'thời gian giao', 'giao nhanh', 'giao chậm', 'phí ship', 'thần tốc','Gói hàng'
]

chat_luong_keywords = [
    'chất lượng', 'bền', 'hỏng', 'tốt', 'xấu', 'đểu', 'kém', 'chắc chắn',
    'dễ vỡ', 'hư', 'dùng tốt', 'sản phẩm tốt', 'hoạt động', 'dùng', 'chất lượng kém','Nội dung','hay','đẹp','xịn','độc','lạ',
    'đẹp trai', 'đẹp gái', 'mới', 'cũ', 'hàng chính hãng', 'hàng giả', 'hàng nhái','Tuyệt vời'
]

gia_ca_keywords = [
    'giá', 'đắt', 'rẻ', 'khuyến mãi', 'hợp lý', 'giá cả', 'giá trị', 'tiền',
    'đáng tiền', 'không đáng tiền', 'sale', 'giảm giá', 'rẻ hơn', 'đắt hơn'
]

cskh_keywords = [
    'nhân viên', 'tư vấn', 'hỗ trợ', 'chăm sóc', 'thái độ', 'phục vụ',
    'nhiệt tình', 'phản hồi', 'chăm sóc khách hàng', 'khách hàng', 'dịch vụ'
]

In [28]:
def contains_keyword(text, keywords):
    if isinstance(text, str):
        text = text.lower()
        for keyword in keywords:
            if keyword in text:
                return True
    return False

def classify_aspect(text):
    aspects = []
    
    if contains_keyword(text, van_chuyen_keywords):
        aspects.append('ship')
        
    if contains_keyword(text, chat_luong_keywords):
        aspects.append('quality')
        
    if contains_keyword(text, gia_ca_keywords):
        aspects.append('price')
        
    if contains_keyword(text, cskh_keywords):
        aspects.append('cskh')
        
    if not aspects:
        return 'other'
    
    return ', '.join(aspects)

In [29]:
df['aspect'] = df['content'].apply(classify_aspect)

print("Phân phối các khía cạnh:")
aspect_counts = df['aspect'].value_counts()
print(aspect_counts)

Phân phối các khía cạnh:
aspect
quality                       2856
ship, quality                 1616
other                         1504
ship                           724
quality, price                 491
ship, quality, price           293
price                          153
ship, quality, cskh             96
quality, cskh                   64
ship, price                     60
ship, quality, price, cskh      37
ship, cskh                      28
quality, price, cskh            24
cskh                            17
price, cskh                      5
ship, price, cskh                5
Name: count, dtype: int64


In [30]:
# Xem một số ví dụ cho mỗi khía cạnh
for aspect in aspect_counts.index[:5]:
    print(f"\n\n--- Ví dụ cho khía cạnh: {aspect} ---")
    sample = df[df['aspect'] == aspect].sample(min(3, len(df[df['aspect'] == aspect])))
    for i, (_, row) in enumerate(sample.iterrows()):
        print(f"Đánh giá {i+1}: {row['content']}")



--- Ví dụ cho khía cạnh: quality ---
Đánh giá 1: Cuốn sách rất hay
Đánh giá 2: Sách siêu hay luôn nha mn!!
Đánh giá 3: sách đẹp nhưng thiếu trang


--- Ví dụ cho khía cạnh: ship, quality ---
Đánh giá 1: Nội dung sách hay. Chưa lần nào mua tiki mà thất vọng cả. Sách mới mới cả hình thức và vừa mới xuất bản 7.2022. Giao nhanh siêu cấp thiên hà chưa đầy 12 tiếng. Shipper thân thiện
Đánh giá 2: giao hàng nhanh. sách đẹp rất hài lòng
Đánh giá 3: Tiki gói hàng ko bao giờ làm mình thất vọng, rất là kĩ càng gọn đẹp, giao cực kì nhanh nữa sáng đặt chiều có ùi


--- Ví dụ cho khía cạnh: other ---
Đánh giá 1: sách có nội dung về kĩ năng mềm vô cùng cần thiết cho gen Z
Đánh giá 2: Không bọc chống sốc, sướt bìa
Đánh giá 3: Ok đó


--- Ví dụ cho khía cạnh: ship ---
Đánh giá 1: Đặt lúc 8h hơn mà gần 11h shiper đã giao rv
Đánh giá 2: Đóng gói cẩn thận giao nhanh
Đánh giá 3: Ngắn gọn,súc tích. Giao hàng nhanh


--- Ví dụ cho khía cạnh: quality, price ---
Đánh giá 1: Tâm lý học về tiền tác giả chia sẻ

In [31]:
output_path = '../Labeled_Data/Simple_Label.csv'
df.to_csv(output_path, index=False)
print(f"Đã lưu dữ liệu đã gán nhãn vào {output_path}")

Đã lưu dữ liệu đã gán nhãn vào ../Labeled_Data/Simple_Label.csv


In [32]:
other_df = df[df['aspect'] == 'other']

print(f"Số lượng đánh giá với nhãn 'other': {len(other_df)}")

print("\nMột số ví dụ đánh giá với nhãn 'other':")
print(other_df.sample(5)['content'])

other_output_path = '../Labeled_Data/Other_Label.csv'
other_df.to_csv(other_output_path, index=False)
print(f"\nĐã lưu các đánh giá có nhãn 'other' vào {other_output_path}")

Số lượng đánh giá với nhãn 'other': 1504

Một số ví dụ đánh giá với nhãn 'other':
3736                      Sản phẩm rất oke nha mọi người.
1341                         Sách healing ! Đọc siêu cuốn
1757    rất cảm động và ý nghĩa, cuộc đời là do mỗi ng...
2950    một chiến lược đầu tư tập trung cao độ, nhà đầ...
4813                                             đáng mua
Name: content, dtype: object

Đã lưu các đánh giá có nhãn 'other' vào ../Labeled_Data/Other_Label.csv
